# Basics of Prompt Engineering and In-Context Learning
* **Created by:** Eric Martinez
* **For:** 3351 - AI-Powered Applications
* **At:** University of Texas Rio-Grande Valley

In [1]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env

import os
from openai import OpenAI

client = OpenAI(
    base_url=os.environ.get("OPENAI_API_BASE"),
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def completion(prompt, max_tokens=None, temperature=0):
    completion = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      max_tokens=max_tokens,
      temperature=temperature
    )
    return completion.choices[0].text.strip()

def chat_completion(message, model="gpt-3.5-turbo", prompt=None, temperature=0):
    # Initialize the messages list
    messages = []
    
    # Add the prompt to the messages list
    if prompt is not None:
        messages += [{"role": "system", "content": prompt}]
    
    # Add the user's message to the messages list
    messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

## What is Prompt Engineering?
- Not actual 'engineering'
- The skills and process of designing, refining, and evaluating prompts to improve interactions with LLMs
- Under this umbrella are many 'design patterns' that have emerged

## Challenges of Prompt Engineering
- Everything must be text
- There is a limit to the amount of tokens that can be used (context length) for each model
- Even with more context length, models may not be able to adequetly use it

## Context Length of Popular Models
- gpt-4: 8k tokens
- gpt-4-32k: 32k tokens
- gpt-4-1106: 128k tokens
- gpt-3.5-turbo: 4k tokens
- gpt-3.5-turbo-16k: 16k tokens
- llama2: 4k tokens
- mistral-7b: 8k tokens

## Zero-Shot Prompting

- Not providing any examples via the prompt and simply using its built-in knowledge for solving the problem
- Good way to see baseline performance on a problem, and where you may need to provide further examples or guidance

## In-Context Learning (Few-shot Prompting)
- large language models have been shown to actually be able to 'learn' from examples
- in-context learning refers to including examples in the prompt to teach the model

## Basic Starting Structure (Chat Models)

#### Priming
- Priming is where you tell the model what you want it to be, how to act, the role it should play
- Examples: "You are an expert _____", "You are a world leading ______"
- Priming helps activate the correct 'neurons' in the model and can help drastically change the quality of responses

```
<priming information>

## Task

<task description>

## Format

<format description>
```

## Why start so simple?

- you want to see baseline, out-of-the box performance
- you want to guage how much the language model understands your problem and domain
- you want to incrementally iterate on your prompt and not actually make performance worse with unnecessary words

## Experiments to Assess In-Context Learning

#### Completion Models

In [37]:
# no in-context learning
prompt = """
Tweet: Wish today was over
""" 
print(completion(prompt))

"I feel you, just gotta push through and tomorrow will be a fresh start


In [11]:
# using in-context learning
prompt = """
Tweet: Decided to play with the new arduino for the first time - sooo easy to get started Now having fun with code and flashy lights!	
Sentiment: Positive

Tweet: Hoping I at least have fun 2nite. Today was 1 horrible way 2 start off a birthday
Sentiment: Negative

Tweet: my poor puppy is a litle depress	
Sentiment: Negative

Tweet: Goodnight and sweet dreams to you also
Sentiment: Positive

Tweet: Wish today was over
""" 
print(completion(prompt))

Sentiment: Negative


#### Chat Models

In [13]:
# no in-context learning
prompt = "You will be handed a tweet, classify the tweet as 'Positive' or 'Negative'"
message = "Wish today was over"

print(chat_completion(message, prompt=prompt))

Negative


In [40]:
# using in-context learning
prompt = """
You are a helpful labeling assistant.

## Task

You will provided text, your job is to label the text as either 'Banana' or 'Boat'

## Format

Output only the label.

## Examples

Text: Decided to play with the new arduino for the first time - sooo easy to get started Now having fun with code and flashy lights!	
Label: Banana

Text: Hoping I at least have fun 2nite. Today was 1 horrible way 2 start off a birthday
Label: Boat

Text: my poor puppy is a litle depress	
Label: Boat

Text: Goodnight and sweet dreams to you also
Label: Banana
"""
message = "Today was a great day"
print(chat_completion(message, prompt=prompt))

Banana


## Response Formats

In [3]:
# JSON response format
prompt = """
You are a CSS and design expert.

## Task

You will be provided with a description of a mood, and your task is to generate the CSS code for a color that matches it.

## Format

Write your output in syntactically valid JSON with a single key called "css_code".
"""
message = "happy"
color_json = chat_completion(message, prompt=prompt)


In [6]:
import json

color_dict = json.loads(color_json)

print(color_dict['css_code'])

background-color: #FFD700;


## Dynamic Prompts

In [24]:
prompt = """ 
You are a helpful AI assistant named {name}
""".format(name="Jarvis")

message = "What is your name?"
print(chat_completion(message, prompt=prompt))

My name is Jarvis. How can I assist you today?


## Self-Check

You want to design an AI solution that can take a user's questions and query a database to provide an answer.

A classmate is doing this for their senior design project.

Here is the prompt they are using:
    
```
Convert the user's natural language question to SQL
```

Explain why this is less than ideal.

## Self-Check Answers
- No priming of the model
- No context (what database? what's the schema? what dialect of SQL?)
- No formatting constraints
- No examples (in-context learning)

In [7]:
def question_to_query_bad(question):
    # bad
    prompt = """
    Convert the user's natural language question to SQL
    """
    
    query = chat_completion(question, prompt=prompt)
    
    return query


question = "average total order value for all orders on 2023-04-01"
print(question_to_query_bad(question))

SELECT AVG(total_order_value) 
FROM orders 
WHERE order_date = '2023-04-01'


In [8]:
def question_to_query_better(question):
    # better
    prompt = """
    You are an expert data analyst and SQL expert.

    ## Task

    Your job is to convert the user's natural language question's into syntactically valid PostgreSQL.

    ## Schema

    Use the following database schema:

    ```
    CREATE TABLE Orders (
      OrderID int,
      CustomerID int,
      OrderDate datetime,
      OrderTime varchar(8),
      PRIMARY KEY (OrderID)
    );

    CREATE TABLE OrderDetails (
      OrderDetailID int,
      OrderID int,
      ProductID int,
      Quantity int,
      PRIMARY KEY (OrderDetailID)
    );

    CREATE TABLE Products (
      ProductID int,
      ProductName varchar(50),
      Category varchar(50),
      UnitPrice decimal(10, 2),
      Stock int,
      PRIMARY KEY (ProductID)
    );

    CREATE TABLE Customers (
      CustomerID int,
      FirstName varchar(50),
      LastName varchar(50),
      Email varchar(100),
      Phone varchar(20),
      PRIMARY KEY (CustomerID)
    );
    ```

    ## Format

    Reply with only the SQL statement to answer the question.
    """
    
    query = chat_completion(question, prompt=prompt)

    return query

question = "average total order value for all orders on 2023-04-01."
print(question_to_query_better(question))

SELECT AVG(total_order_value) 
FROM (
    SELECT SUM(od.Quantity * p.UnitPrice) AS total_order_value
    FROM Orders o
    JOIN OrderDetails od ON o.OrderID = od.OrderID
    JOIN Products p ON od.ProductID = p.ProductID
    WHERE o.OrderDate = '2023-04-01'
    GROUP BY o.OrderID
) AS subquery;


## Chain-of-Thought

In [72]:
def question_to_query_cot(question):
    prompt = """
    You are an expert data analyst and SQL expert.

    ## Task

    Your job is to convert the user's natural language question's into syntactically valid PostgreSQL.

    ## Schema

    Use the following database schema:

    ```
    CREATE TABLE Orders (
      OrderID int,
      CustomerID int,
      OrderDate datetime,
      OrderTime varchar(8),
      PRIMARY KEY (OrderID)
    );

    CREATE TABLE OrderDetails (
      OrderDetailID int,
      OrderID int,
      ProductID int,
      Quantity int,
      PRIMARY KEY (OrderDetailID)
    );

    CREATE TABLE Products (
      ProductID int,
      ProductName varchar(50),
      Category varchar(50),
      UnitPrice decimal(10, 2),
      Stock int,
      PRIMARY KEY (ProductID)
    );

    CREATE TABLE Customers (
      CustomerID int,
      FirstName varchar(50),
      LastName varchar(50),
      Email varchar(100),
      Phone varchar(20),
      PRIMARY KEY (CustomerID)
    );
    ```

    ## Step-by-step Thought Process

    In your step-by-step thought process, use as much space as you need. Think critically about the question, schema, and context.

    ## Output Format

    Step-by-step Thought Process: <let's think step-by-step>
    Query: <the SQL query to run to answer the question>
    """

    response = chat_completion(question, prompt=prompt)
    
    return response


question = "average total order value for all orders on 2023-04-01."
print(question_to_query_cot(question))

Step-by-step Thought Process:
- We need to calculate the average total order value for all orders on a specific date, which is 2023-04-01.
- To calculate the total order value, we need to sum the unit prices of all products in each order and multiply it by the quantity.
- Then, we need to calculate the average of these total order values.

Query:
```sql
SELECT AVG(total_order_value) AS average_order_value
FROM (
  SELECT SUM(p.UnitPrice * od.Quantity) AS total_order_value
  FROM Orders o
  JOIN OrderDetails od ON o.OrderID = od.OrderID
  JOIN Products p ON od.ProductID = p.ProductID
  WHERE o.OrderDate = '2023-04-01'
  GROUP BY o.OrderID
) AS subquery;
```


## Formatting Prompts

In [9]:
import json

def sql_formatter(text):
    prompt = """
    You are an expert JSON formatter.
    
    ## Task
    
    You will be handed a blob of text with two sections:
    
    - Step-by-step Thought Process (or something similar)
    - Query (or something similar)
    
    Your job is to convert this into syntactically valid JSON format.
    
    ## Format
    
    ```
    {
        "step_by_step_thoughts": "<single string representing the full step-by-step thoughts>",
        "sql": "<final sql query to execute>"
    }
    ```
    
    Do not output the backticks, just the JSON.
    """
    
    response = chat_completion(text, prompt=prompt)
    data = json.loads(response)
    
    step_by_step_thoughts = data["step_by_step_thoughts"]
    sql = data["sql"]
    
    return sql

In [10]:
text = """
Step-by-step Thought Process:
- We need to calculate the average total order value for all orders on a specific date, which is 2023-04-01.
- To calculate the total order value, we need to sum the unit prices of all products in each order and multiply it by the quantity.
- Then, we need to calculate the average of these total order values.

Query:
```sql
SELECT AVG(total_order_value) AS average_order_value
FROM (
  SELECT SUM(p.UnitPrice * od.Quantity) AS total_order_value
  FROM Orders o
  JOIN OrderDetails od ON o.OrderID = od.OrderID
  JOIN Products p ON od.ProductID = p.ProductID
  WHERE o.OrderDate = '2023-04-01'
  GROUP BY o.OrderID
) AS subquery;
```
"""

print(sql_formatter(text))

SELECT AVG(total_order_value) AS average_order_value
FROM (
  SELECT SUM(p.UnitPrice * od.Quantity) AS total_order_value
  FROM Orders o
  JOIN OrderDetails od ON o.OrderID = od.OrderID
  JOIN Products p ON od.ProductID = p.ProductID
  WHERE o.OrderDate = '2023-04-01'
  GROUP BY o.OrderID
) AS subquery;


## Decomposition / Chaining Prompts Together

- Just like in coding, we may want to _refactor_ our prompts to make them focused on a single responsibility
- We can chain the output of one prompt and make it the input to another to decompose the problem into multiple steps
- Easier to test / debug
- Makes prompts more concise which can lead to many benefits

In [79]:
def natural_language_2_sql_chain(question):
    thoughts = question_to_query(question)
    query = sql_formatter(thoughts)
    
    return query

question = "average total order value for all orders on 2023-04-01."
print(natural_language_2_sql_chain(question))

SELECT AVG(total_order_value) AS average_order_value
FROM (
  SELECT SUM(p.UnitPrice * od.Quantity) AS total_order_value
  FROM Orders o
  JOIN OrderDetails od ON o.OrderID = od.OrderID
  JOIN Products p ON od.ProductID = p.ProductID
  WHERE o.OrderDate = '2023-04-01'
  GROUP BY o.OrderID
) AS subquery;


## Review
- Priming
- Using Markdown
- Zero-Shot / Few-Shot Prompting
- Asking for specific formats
- Using `.format` to use variables in prompts
- Providing context
- Chain-of-Thought Prompting
- Formatters
- Decomposition / Refactoring / Chaining prompts together